In [1]:
## Ran for eveything now with excluding the gauges that we don't want

# Create a set of dimensionless profiles
Read in all of the events, for all durations, for all gauges, for all ensemble members.  
Convert them to dimensionless profiles, with 12 values between 0 and 1.  
Each value is a dimensionless, cumulative rainfall value (cumulative rainfall at this timestep, normalised by the total event rainfall):
- 0 means no rainfall has occurred, and 
- 1 means the total event rainfall has been reached.  

If there are less than 12 values, then these are filled in with interpolation

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import re
from datetime import datetime

from Create_Profiles_Functions import *

quintile_mapping = {1: 'F2', 2: 'F1', 3: 'C', 4: 'B1', 5: 'B2'}

In [9]:
def find_part_with_most_rain(array, n, plot=False, ax=False):
    if array is None:
        return None
    else:

        # Compute differences
        # Split the array into 5 equal parts
        splits = np.array_split(array, n)

        max_array_rainfall = 0
        max_array_num = None

        total_precipitations = []  # To store total precipitation for each split
        split_ranges = []  # To store start and end indices for each split

        # Calculate total precipitation for each split
        split_start = 0
        for split in splits:
            total_precipitation = split.sum()
            total_precipitations.append(total_precipitation)
            split_end = split_start + len(split)
            split_ranges.append((split_start, split_end))
            if total_precipitation > max_array_rainfall:
                max_array_num = len(total_precipitations)
                max_array_rainfall = total_precipitation
            split_start = split_end

        colors = ['lightblue'] * n  # Default color for all splits
        highlight_color = 'yellow'  # Color for the split with the most rainfall

        if plot:
            # Plot the array
            ax.plot(range(1, len(array) + 1), array, label='Precipitation', marker='o')

            # Add vertical lines and shading for each split segment
            for i, (start_index, end_index) in enumerate(split_ranges):
                color = highlight_color if (i + 1) == max_array_num else colors[i]

                # Add vertical lines at the start and end of each split
                ax.axvline(x=start_index + 1, color=color, linestyle='--', label=f'Split {i+1} Start' if i == 0 or (i + 1) == max_array_num else "")
                ax.axvline(x=end_index, color=color, linestyle='--', label=f'Split {i+1} End' if i == 0 or (i + 1) == max_array_num else "")

                # Shade the region for the split
                ax.fill_between(range(start_index + 1, end_index + 1), array[start_index:end_index], color=color, alpha=0.3)

                # Add the total precipitation value behind the shading
                ax.text((start_index + end_index) / 2+0.5, max(array) * 0.05,  # Adjust y-position if needed
                        f'{total_precipitations[i]:.2f}',
                        ha='center', va='center', fontsize=10, color='black', weight='bold', zorder=1)

            ax.set_title(f'Precipitation Values with Splits Marked. Max at {max_array_num}')
            ax.set_xlabel('Time')
            ax.set_ylabel('Precipitation')

    return max_array_num


def create_normalised_event(rainfall):
    # Check if the input array is None or empty
    if rainfall is None or len(rainfall) == 0:
        # print("Input array is None or empty. Cannot normalize.")
        return None

    # Check if the maximum value is zero to avoid division by zero
    if np.max(rainfall) == 0:
        print("Maximum rainfall is zero. Cannot normalize.")
        return rainfall  # Return the input as-is, or handle appropriately

    # Normalize rainfall from 0 to 1 using the maximum value
    normalized_rainfall = rainfall / np.max(rainfall)
    normalized_rainfall = normalized_rainfall.to_list()
    # Debug prints to check the input and output
    return normalized_rainfall

def create_cumulative_event(rainfall):
    
    # Calculate cumulative rainfall
    cumulative_rainfall = np.cumsum(rainfall)
    
    return cumulative_rainfall.tolist()

def interpolate_rainfall(rainfall, bin_number):
    if rainfall is None or len(rainfall) < 2:
        return None

    # Define target points for bin_number bins
    target_points = np.linspace(0, 1, bin_number)
    
    # Create interpolation function based on existing data points
    rainfall_times = np.array(range(0, len(rainfall)))

    # Normalize time from 0 to 1
    normalized_time = (rainfall_times - rainfall_times[0]) / (rainfall_times[-1] - rainfall_times[0])
    interpolation_func = interp1d(normalized_time, rainfall, kind='linear', fill_value="extrapolate")
    
    # Interpolate values at target points
    interpolated_values = interpolation_func(target_points)
    
    return interpolated_values


def create_incremental_event(cumulative_rainfall):
    if cumulative_rainfall is None :
        return None
    
    raw_rainfall = np.diff(cumulative_rainfall, prepend=0)
    raw_rainfall[0] = cumulative_rainfall[0]
    return raw_rainfall

def extract_year(df):
    # Ensure the 'times' column is in datetime format
    df['times'] = pd.to_datetime(df['times'], errors='coerce')  # errors='coerce' will handle invalid parsing
    # Extract the year
    return df['times'].dt.year[0]

In [3]:
gauge_nums = range(0,1294)

# NIMROD - 30 mins
### Make profiles

In [ ]:
# # Initialize an empty DataFrame with the desired columns
columns = [
    'gauge_num',  'season', 'precip', 'Volume','Year', 'times', 'duration',
    'normalized_rainfall', 'normalized_interpolated_rainfall_12', 'normalized_interpolated_rainfall_15',
    'max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_raw_rain']
df = pd.DataFrame(columns=columns)

for nimrod_option in ["NIMROD_2.2km_filtered_100"]:
    for gauge_num in range(0, 1293):
        if gauge_num not in [444, 827, 888]:
            print(f"gauge {gauge_num}")
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
            files = np.sort(files)

            for file in files:
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    continue

                this_event = read_event(gauge_num, fp)
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                if real_trimmed_event is not None:
                    
                    precip = real_trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = create_normalised_event(real_trimmed_event['precipitation (mm/hr)'])
                    cumulative_normalized_rainfall = create_cumulative_event(normalized_rainfall)
                    interpolated15_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,15)
                    interpolated12_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,12)
                    interpolated15_incremental_normalized_rainfall = create_incremental_event(interpolated15_cumulative_normalized_rainfall)
                    interpolated12_incremental_normalized_rainfall = create_incremental_event(interpolated12_cumulative_normalized_rainfall)

                    max_quintile_profile_12 = find_part_with_most_rain(interpolated12_incremental_normalized_rainfall, 5)
                    max_quintile_profile_15 = find_part_with_most_rain(interpolated15_incremental_normalized_rainfall, 5)
                    max_quintile_normalised_rain = find_part_with_most_rain(normalized_rainfall, 5)
                    max_quintile_raw_rain = find_part_with_most_rain(precip, 5)
                    
                    duration = len(real_trimmed_event) / 2
                    times = trimmed_event['times']
                    season = get_season(trimmed_event['times'][0])
                    year=extract_year(trimmed_event)
                    
                else:
                    precip=trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = None
                    normalized_interpolated_rainfall_12 = None
                    normalized_interpolated_rainfall_15 = None
                    max_quintile_profile_12 = None
                    max_quintile_profile_15 = None
                    max_quintile_normalised_rain = None
                    max_quintile_raw_rain = None
                    duration = None
                    season = None
                    year=None
                    times=None
                    
                # Append the row to the DataFrame
                df = df.append({
                    'gauge_num': gauge_num,
                    'season': season,
                    'precip': precip.values,
                    'Volume': sum(precip),
                    'Year':year,
                    'times':times, 
                    'duration': duration,
                    'normalized_rainfall': normalized_rainfall,
                    'normalized_interpolated_rainfall_12': interpolated12_incremental_normalized_rainfall,
                    'normalized_interpolated_rainfall_15': interpolated15_cumulative_normalized_rainfall,
                    'max_quintile_profile_12': max_quintile_profile_12,
                    'max_quintile_profile_15': max_quintile_profile_15,
                    'max_quintile_normalised_rain': max_quintile_normalised_rain,
                    'max_quintile_raw_rain': max_quintile_raw_rain
                }, ignore_index=True)
                
df['Loading_profile12'] = df['max_quintile_profile_12'].map(quintile_mapping)
df['Loading_profile15'] = df['max_quintile_profile_15'].map(quintile_mapping)
df['Loading_profile_normalised_rain'] = df['max_quintile_normalised_rain'].map(quintile_mapping)
df['Loading_profile_raw_rain'] = df['max_quintile_raw_rain'].map(quintile_mapping)

with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/{nimrod_option}_profiles_df.pkl", 'wb') as file:
    pickle.dump(df, file)

gauge 0
gauge 1
gauge 2
gauge 3
gauge 4


# NIMROD 5 mins

In [112]:
for nimrod_option in ["NIMROD_1km_filtered_300"]:

    # New list to store just the files that are 'good' e.g. 
    dict_of_things_im_storing ={}
    dict_of_things_im_storing['good_raw_events'] = []
    dict_of_things_im_storing['good_raw_events_seasons'] = []
    dict_of_things_im_storing['good_trimmed_events'] = []
    dict_of_things_im_storing['good_trimmed_events_durations'] = []
    dict_of_things_im_storing['good_normalised_events'] = []
    dict_of_things_im_storing['good_normalised_interpolated_events'] = []
    dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'] = []

    for gauge_num in range(0,1262):
        if gauge_num not in [443, 444, 827, 888]:
            print(gauge_num)

            # Create a list of all the event CSVs
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
            files = np.sort(files)

            # Go through each event
            for file in files:
                # Construct the path to the file
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
                # Read in event
                this_event = read_event(gauge_num, fp)
                # Trim leading and trailing zeroes
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)

                # Check trimmed event for 'problems', if there are none, return a dataframe, if there are, return None
                trimmed_event = remove_events_with_problems(trimmed_event, verbose=True)

                # If we still have a dataframe, then add it to list of good, unprocessed events
                if trimmed_event is not None:

                    # Create normalised version of event
                    normalized_time, normalized_rainfall = create_normalised_event(trimmed_event)
                    # Create interpolated, normalised version of event (with 12 data points)
                    normalized_interpolated_rainfall = interpolate_and_bin(normalized_time, normalized_rainfall)
                    # Find the fifth OF THE INTERPOLATED, NORMALISED EVENT with the maximum amount of rainfall
                    max_quintile = find_fifth_with_most_rain(normalized_interpolated_rainfall)

                    # Add to list of events with no problems
                    dict_of_things_im_storing['good_raw_events'].append(this_event)
                    dict_of_things_im_storing['good_raw_events_seasons'].append(get_season(this_event['times'][0]))
                    dict_of_things_im_storing['good_trimmed_events'].append(trimmed_event)
                    dict_of_things_im_storing['good_trimmed_events_durations'].append(len(trimmed_event)/12)
                    dict_of_things_im_storing['good_normalised_events'].append(normalized_rainfall)
                    dict_of_things_im_storing['good_normalised_interpolated_events'].append(normalized_interpolated_rainfall)
                    dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'].append(max_quintile)

    for file_name,stored_list in dict_of_things_im_storing.items():
        print(file_name)
        cache_filepath = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_5mins/WholeYear/{nimrod_option}_{file_name}.pkl"
        with open(cache_filepath, 'wb') as f:
            pickle.dump(stored_list, f)                    

0
1
2
3
4
5
Doesn't contain more than 1 value which isn't 0
6
7
8
Doesn't contain more than 1 value which isn't 0
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Doesn't contain more than 1 value which isn't 0
31
32
33
34
35
36
Doesn't contain more than 1 value which isn't 0
Doesn't contain more than 1 value which isn't 0
37
38
39
40
41
42
Doesn't contain more than 1 value which isn't 0
43
Doesn't contain more than 1 value which isn't 0
44
Too short to be an event
45
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
46
47
48
49
50
51
Doesn't contain more than 1 value which isn't 0
Too short to be an event
52
53
54
Doesn't contain more than 1 value which isn't 0
55
56
57
58
59
60
61
62
63
64
65
66
Doesn't contain more than 1 value which isn't 0
67
68
69
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an e

1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
good_raw_events
good_raw_events_seasons
good_trimmed_events
good_trimmed_events_durations
good_normalised_events
good_normalised_interpolated_events
good_normalised_interpolated_events_max_quintiles


# UKCP18
### Make and pickle profiles

In [15]:
# # Initialize an empty DataFrame with the desired columns
# Initialize an empty DataFrame with the desired columns
columns = [
    'gauge_num',  'season', 'precip', 'Volume','Year', 'times', 'duration',
    'normalized_rainfall', 'cumulative_normalized_rainfall', 'cumulative_rainfall',
    'interpolated15_cumulative_rainfall', 'interpolated12_cumulative_rainfall','interpolated15_incremental_rainfall','interpolated12_incremental_rainfall',
    'interpolated15_cumulative_normalized_rainfall','interpolated12_cumulative_normalized_rainfall','interpolated15_incremental_normalized_rainfall','interpolated12_incremental_normalized_rainfall',
    'max_quintile_raw_rain','max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_profile_12_norm', "max_quintile_profile_15_norm"]
df2 = pd.DataFrame(columns=columns)

for em in ["bb189"]:
    for gauge_num in range(148, 1293):
        if gauge_num not in [444, 827, 888]:
            print(f"gauge {gauge_num}")
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/") if f.endswith('.csv')]
            files = np.sort(files)

            for file in files:
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    continue

                this_event = read_event(gauge_num, fp)
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                if real_trimmed_event is not None:
                    
                    precip = real_trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = create_normalised_event(real_trimmed_event['precipitation (mm/hr)'])
                    cumulative_normalized_rainfall = create_cumulative_event(normalized_rainfall)
                    cumulative_rainfall = create_cumulative_event(precip)
                    
                    interpolated15_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,15)
                    interpolated12_cumulative_rainfall = interpolate_rainfall(cumulative_rainfall,12)
                    interpolated15_incremental_rainfall = create_incremental_event(interpolated15_cumulative_rainfall)
                    interpolated12_incremental_rainfall = create_incremental_event(interpolated12_cumulative_rainfall)
                    
                    interpolated15_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,15)
                    interpolated12_cumulative_normalized_rainfall = interpolate_rainfall(cumulative_normalized_rainfall,12)
                    interpolated15_incremental_normalized_rainfall = create_incremental_event(interpolated15_cumulative_normalized_rainfall)
                    interpolated12_incremental_normalized_rainfall = create_incremental_event(interpolated12_cumulative_normalized_rainfall)
                    
                    max_quintile_raw_rain = find_part_with_most_rain(precip, 5)
                    max_quintile_profile_12 = find_part_with_most_rain(interpolated12_incremental_rainfall, 5)
                    max_quintile_profile_15 = find_part_with_most_rain(interpolated15_incremental_rainfall, 5)
                    
                    max_quintile_normalised_rain = find_part_with_most_rain(normalized_rainfall, 5)
                    max_quintile_profile_12_norm = find_part_with_most_rain(interpolated12_incremental_normalized_rainfall, 5)
                    max_quintile_profile_15_norm = find_part_with_most_rain(interpolated15_incremental_normalized_rainfall, 5)
                    
                    duration = len(real_trimmed_event) / 2
                    times = trimmed_event['times'].values
                    season = get_season(trimmed_event['times'][0])
                    year=extract_year(trimmed_event)
                    
                else:
                    precip=trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = None
                    cumulative_normalized_rainfall = None
                    cumulative_rainfall = None
                    
                    interpolated15_cumulative_rainfall = None
                    interpolated12_cumulative_rainfall = None
                    interpolated15_incremental_rainfall = None
                    interpolated12_incremental_rainfall = None
                    
                    interpolated15_cumulative_normalized_rainfall = None
                    interpolated12_cumulative_normalized_rainfall = None
                    interpolated15_incremental_normalized_rainfall = None
                    interpolated12_incremental_normalized_rainfall = None
                    
                    max_quintile_raw_rain = None
                    max_quintile_profile_12 = None
                    max_quintile_profile_15 = None
                    
                    max_quintile_normalised_rain = None
                    max_quintile_profile_12_norm = None
                    max_quintile_profile_15_norm = None
                    
                    duration = None
                    season = None
                    year=None
                    times=None
                    
                # Append the row to the DataFrame
                df2 = df2.append({
                    'gauge_num': gauge_num,
                    'season': season,
                    'precip': precip.values,
                    
                    'Volume': sum(precip),
                    'Year':year,
                    'times':times, 
                    'duration': duration,
                    
                    'normalized_rainfall':normalized_rainfall,
                    'cumulative_normalized_rainfall':cumulative_normalized_rainfall,
                    'cumulative_rainfall':cumulative_rainfall,
                    
                    'interpolated15_cumulative_rainfall':interpolated15_cumulative_rainfall,
                    'interpolated12_cumulative_rainfall':interpolated12_cumulative_rainfall,                    
                    'interpolated15_incremental_rainfall':interpolated15_incremental_rainfall,
                    'interpolated12_incremental_rainfall': interpolated12_incremental_rainfall,
                    
                    'interpolated15_cumulative_normalized_rainfall':interpolated15_cumulative_normalized_rainfall,
                    "interpolated12_cumulative_normalized_rainfall":interpolated12_cumulative_normalized_rainfall,
                    'interpolated15_incremental_normalized_rainfall':interpolated15_incremental_normalized_rainfall,
                    'interpolated12_incremental_normalized_rainfall':interpolated12_incremental_normalized_rainfall,
                    
                    'max_quintile_raw_rain': max_quintile_raw_rain,
                    'max_quintile_profile_12': max_quintile_profile_12,
                    'max_quintile_profile_15': max_quintile_profile_15,
                    'max_quintile_normalised_rain': max_quintile_normalised_rain,
                    'max_quintile_profile_12_norm': max_quintile_profile_12_norm,
                    'max_quintile_profile_15_norm': max_quintile_profile_15_norm                    
                }, ignore_index=True)
                

df2['Loading_profile_raw_rain'] = df2['max_quintile_raw_rain'].map(quintile_mapping)
df2['Loading_profile12'] = df2['max_quintile_profile_12'].map(quintile_mapping)
df2['Loading_profile15'] = df2['max_quintile_profile_15'].map(quintile_mapping)
df2['Loading_profile_normalised_rain'] = df2['max_quintile_normalised_rain'].map(quintile_mapping)
df2['Loading_profile_norm_12'] = df2['max_quintile_profile_12_norm'].map(quintile_mapping)
df2['Loading_profile_norm_15'] = df2['max_quintile_profile_15_norm'].map(quintile_mapping)

df2 = add_duration_cats_based_on_data(df2)
df2 = add_duration_cats_predetermined(df2)

# with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/profiles_df.pkl", 'wb') as file:
#     pickle.dump(df2, file)

gauge 148
gauge 149
Some dates were invalid and have been coerced to NaT:
gauge 150
gauge 151
gauge 152
Some dates were invalid and have been coerced to NaT:
gauge 153
gauge 154
gauge 155
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 156
gauge 157
gauge 158
gauge 159
gauge 160
Some dates were invalid and have been coerced to NaT:
gauge 161
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 162
gauge 163
gauge 164
gauge 165
gauge 166
gauge 167
gauge 168
Some dates were invalid and have been coerced to NaT:
gauge 169
gauge 170
Some dates were invalid and have been coerced to NaT:
gauge 171
Some dates were invalid and have been coerced to NaT:
gauge 172
Some dates were invalid and have been coerced to NaT:
gauge 173
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 174
gauge 175
gauge 176
gauge 17

Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 591
gauge 592
gauge 593
gauge 594
gauge 595
gauge 596
gauge 597
gauge 598
gauge 599
gauge 600
gauge 601
gauge 602
gauge 603
gauge 604
gauge 605
gauge 606
gauge 607
gauge 608
gauge 609
gauge 610
gauge 611
gauge 612
gauge 613
gauge 614
gauge 615
gauge 616
gauge 617
gauge 618
gauge 619
gauge 620
gauge 621
gauge 622
gauge 623
gauge 624
gauge 625
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 626
gauge 627
gauge 628
gauge 629
gauge 630
gauge 631
gauge 632
gauge 633
gauge 634
gauge 635
gauge 636
gauge 637
Some dates were invalid and have been coerced to NaT:
gauge 638
gauge 639
gauge 640
gauge 641
gauge 642
gauge 643
gauge 644
gauge 645
gauge 646
gauge 647
gauge 648
gauge 649
gauge 650
gauge 651
gauge 652
gauge 653
gauge 654
gauge 655
gauge 656
gauge 657
gauge 

Some dates were invalid and have been coerced to NaT:
gauge 1059
gauge 1060
Some dates were invalid and have been coerced to NaT:
gauge 1061
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1062
Some dates were invalid and have been coerced to NaT:
gauge 1063
gauge 1064
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1065
Some dates were invalid and have been coerced to NaT:
gauge 1066
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1067
gauge 1068
Some dates were invalid and have been coerced to NaT:
gauge 1069
Some dates were invalid and have been coerced to NaT:
gauge 1070
gauge 1071
gauge 1072
gauge 1073
Some dates were invalid and have been coerced to NaT:
gauge 1074
gauge 1075
Som

gauge 1217
Some dates were invalid and have been coerced to NaT:
gauge 1218
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1219
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1220
gauge 1221
gauge 1222
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1223
Some dates were invalid and have been coerced to NaT:
gauge 1224
Some dates were invalid and have been coerced to NaT:
gauge 1225
Some dates were invalid and have been coerced to NaT:
gauge 1226
gauge 1227
Some dates were invalid and have been coerced to NaT:
Some dates were invalid and have been coerced to NaT:
gauge 1228
gauge 1229
gauge 1230
gauge 1231
Some dates were invalid a

In [16]:
with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/profiles_df.pkl", 'wb') as file:
    pickle.dump(df2, file)